In [2]:
!pip3 install dynamax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#!rm -r Composers

In [4]:
#!tar -xvf Composer_csvs.zip # command to unzip the zip file

In [5]:
from sklearn.model_selection import train_test_split
from dynamax.hidden_markov_model import CategoricalHMM
from sklearn.neighbors import NearestCentroid
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import jax.numpy as jnp
import jax.random as jr
from jax import vmap
from matplotlib import pyplot as plt
import os
import pickle

In [6]:
def csv_to_numpy(composer,song):
  main_dir = "/content/Composers"
  df = pd.read_csv(main_dir + '/' + composer + '-output-csvs' + '/' + song)
  return df[['note_num','normed_duration']].to_numpy()

In [7]:
def load_dictionaries(files = ['train.pkl','test.pkl']):
  train_file = files[0]
  test_file = files[1]
  print(train_file)
  with open(train_file,'rb') as f:
    train_dict = pickle.load(f)
  with open(test_file,'rb') as f:
    test_dict = pickle.load(f)

  return train_dict, test_dict

In [8]:
train_dict, test_dict = load_dictionaries()

train.pkl


In [9]:
def get_train_test():
  main_dir = "/content/Composers"
  train_dict = {}
  test_dict = {}

  # NOTE: deleted debussy and balakir for lack of data
  percent_split = 0.8

  for file_name in os.listdir(main_dir):
    comp_list = []
    if file_name[0] == '.':
      continue
    for song in os.listdir(main_dir + '/' + file_name):
      comp_list.append(song)
    np.random.shuffle(comp_list)  # shuffle list
    # manual train-test split
    len_dir = len(comp_list)
    eighty = int(len_dir*percent_split)
    if eighty < 1:
      raise ValueError("Need to rewrite percentsplit!")

    train_dict[file_name.split('-')[0]] = comp_list[:eighty]
    test_dict[file_name.split('-')[0]] = comp_list[eighty:]

    filename = "/content/train.pkl"
  with open(filename, 'wb') as f:
    pickle.dump(train_dict, f)

  filename = "/content/test.pkl"
  with open(filename, 'wb') as f:
    pickle.dump(test_dict, f)


In [10]:
def print_params(params):
    jnp.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
    print("initial probs:")
    print(params.initial.probs)
    print("transition matrix:")
    print(params.transitions.transition_matrix)
    print("emission probs:")
    print(params.emissions.probs) 

Now Let's work with the test set

In [17]:
def ComposerAnalysis(comps_list, train_dict = train_dict, test_dict = test_dict,verbose = True,emissions=2,states=3):
  """function to analyze a list of composers using provided data in a folder at the directory:
  /content/Composers. Each composers song data should be saved as a csv of the format outputted
  by the midi-csv submodule script titled midi_to_csv.py.
  PARAMETERS:

    comps_list (list): list of the names of composers to run the analysis on as they appear in the
        train dictionary and test dictionary
    train_dict (dictionary): dictionary of composer names to a list of the names of the csvs that contain
        the composer's training songs
    test_dict (dictionary): dictionary of composer names to a list of the names of the csvs that contain that
        composer's test songs
    verbose (bool): whether or not to alert user to progress of program

  RETURNS:
    NONE, but prints the scoring of the test set for Nearest Neighbors and K-Means
    """
  main_dir = "/content/Composers"
  # create labels to be used to identify composers
  comp_labels = {composer:i for i, composer in enumerate(comps_list)}

  # 1) go through all the songs we are training on to get the number of unique notes and durations
  # these will be used in the construction of the HMM model
  notes = set()
  durations = set()
  test_songs = 0
  train_songs = 0

  for composer in comps_list:
    for song in test_dict[composer]:
      test_songs += 1
      arr = csv_to_numpy(composer,song)
      notes.update(arr[:,0])
      durations.update(arr[:,1])
    for song in train_dict[composer]:
      train_songs += 1
      arr = csv_to_numpy(composer,song)
      notes.update(arr[:,0])
      durations.update(arr[:,1])

  if verbose:
    print(f"The total number of notes found was:\t\t{len(notes)}\nThe "
    f"total number of durations found was:\t{len(durations)}\n\n")

  # 2) Train the an HMM model for each song to get the transition matrix for each song

  num_unique_notes = len(notes)                             # for calculating classes
  num_unique_durations = len(durations)                     # for calculating classes
  num_states = states                                       # dimension of hidden state must be more than num_emmisions. eventually we will grid search this
  num_emmisions = emissions                                 # dimension of observations, two because we have note value and durations
  num_classes = num_unique_notes*num_unique_durations       # total number of values the observation can take on 
  param_list_train = []                                     # place to store the parameter objects
  train_labels = []                                         # labels to now what song was from what composer for
                                                            #   clustering later
  # now training
  if verbose:
    print("NOW STARTING TRAINING\n")
    print(f"\tTotal training songs: {train_songs}\n")

  song_progress = 1

  for composer in comps_list:
    for song in train_dict[composer]:
      if verbose:
        print(f"processing song {song_progress}/{train_songs}")
      song_progress += 1
      train_labels.append(comp_labels[composer])
      arr = csv_to_numpy(composer,song)
      if (np.isnan(arr).any()) or (np.isinf(arr).any()):
          print("NaN or Inf")
          print(np.isnan(arr).any())
          print(np.isinf(arr).any())
          print(" at i =", i)
          continue
      hmm = CategoricalHMM(num_states,num_emmisions,num_classes)   # create object
      params, props = hmm.initialize(method="prior")  
      params, log_probs = hmm.fit_em(params,props,arr,num_iters=10)     # fiting training points
      param_list_train.append(params)                                   # saving parameters for that song

  # 3) Processing training data to be used in training classification models
  print("PROCESSING TRAINING DATA\n")

  train_Fmatrix_list = []
  for params in param_list_train:                                       # extracting transition matrices (F matrices)
    train_Fmatrix_list.append(params.transitions.transition_matrix)

  X_for_training = []   
  for matrix in train_Fmatrix_list:                                          # raveling transition matrices so that each
      X_for_training.append(np.ravel(matrix))                           # can act as a point in the clustering algorithm


  # 4) Training classification models with processed training data
  if verbose:
    print("TRAINING CLASSIFICATION MODELS\n")

  NC = NearestCentroid()                                                # nearest neighbors classifier
  NC.fit(np.array(X_for_training), train_labels)
  KM = KMeans(len(comps_list))                                                        # KMeans classifier
  KM.fit(np.array(X_for_training), train_labels)
  RF = RandomForestClassifier()
  RF.fit(np.array(X_for_training), train_labels)


  # 5) Get the test data by getting transition matrices by running the HMM model on the test points

  print(f"STARTING TEST DATA GENERATION\n")
  print(f"\tTotal test songs: {test_songs}\n")

  param_list_test = []  
  test_labels = []
  song_progress = 0           # to keep track of how much progress in the for loop you've done

  for composer in comps_list:
    for song in test_dict[composer]:
      if verbose:
        print(f"processing test song {song_progress}/{test_songs}")
      song_progress += 1
      test_labels.append(comp_labels[composer])
      arr = csv_to_numpy(composer,song)
      if (np.isnan(arr).any()) or (np.isinf(arr).any()):
          print("NaN or Inf")
          print(np.isnan(arr).any())
          print(np.isinf(arr).any())
          print(" at i =", i)
          continue
      hmm = CategoricalHMM(num_states,num_emmisions,num_classes)
      params, props = hmm.initialize(method="prior")
      params, log_probs = hmm.fit_em(params,props,arr,num_iters=10)
      param_list_test.append(params)

  if verbose:
    print("PROCESSING TEST DATA\n")

  t_matrix_list_test = []
  for params in param_list_test:                # raveling for clustering purposes
      t_matrix_list_test.append(params.transitions.transition_matrix)
  
  X_for_scoring = []                            # place to store ravelled t-matrices of test data for scoring
  for matrix in t_matrix_list_test:
      X_for_scoring.append(np.ravel(matrix))
  
  #return X_for_training, X_for_scoring

  # 6) # Get scores based on the test data
  if verbose:
    print("SCORING PROCESSED TEST DATA\n")

  # nearest neighbors score
  NN_score = NC.score(np.array(X_for_scoring), test_labels)

  # K-means score
  pred_labels = KM.predict(np.array(X_for_scoring))
  KM_score = accuracy_score(test_labels,pred_labels)

  # RF score
  RF_labels = RF.predict(X_for_scoring)
  RF_score = accuracy_score(test_labels,RF_labels)

  print(f"RESULTS:\n\nNearest Neighbors Score:\t{NN_score}\nK-Means Score:\t\t{KM_score}\nRF Score:\t\t{RF_score}")


In [18]:
#comps_list = ['haydn','mozart','schubert']
#ComposerAnalysis(comps_list)

In [19]:
new_list = ['chopin','borodin','beeth','liszt','mendelssohn','schumann']

In [20]:
#ComposerAnalysis(new_list)

In [21]:
test_list = ['mozart','chopin','haydn']

In [22]:
for s in [12]:
  ComposerAnalysis(test_list,states=s,verbose=False)
  print(f"state dimension: {s}")

PROCESSING TRAINING DATA



/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


STARTING TEST DATA GENERATION

	Total test songs: 13



RESULTS:

Nearest Neighbors Score:	0.3076923076923077
K-Means Score:		0.38461538461538464
RF Score:		0.5384615384615384
state dimension: 12
